# Getting Started

In this tutorial, we will learn about LangChain's concise API. We will cover:
- Using the `generate`, `decide`, and `choice` functions to generate text, construct python objects, make decision, and select options.
- Using the `template` and `gemplate` functions to create reusable text templators and semantic kernels.
- Using `rulex` to perform semantic pattern matching and replacement.

The `langchain.concise` submodule contains the following functions
- choice.py  which provides a function for choosing an option from a list of options based on a query and examples.
- chunk.py  which splits text into smaller chunks.
- config.py  which provides functions for setting and getting default values for the language model, text splitter, and maximum tokens.
- decide.py  which determines whether a statement is true or false based on a query and examples.
- gemplate.py which defines a function for creating reusable semantic kernels.
- generate.py  which generates text using a language model and provides convenience options for parsing, removing quotes, and retrying failed attempts.
- pattern.py which provides a function for prompting an LLM to complete a pattern.
- rulex.py  which provides a class for defining natural language replacement rules for text.
- template.py  which defines a function for creating reusable text templators.

## Why do we need a concise API?

The concise API provides many one-liners for common use cases. For example, before the concise API existed, generating a templated completion involved several steps:
1. creates a prompt template,
2. renders it into a prompt,
3. formats it with variables,
4. constructs an LLM,
5. calls the llm with the formatted prompt
6. and returns the result.

In [10]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

prompt_template = 'Hello, my name is {name}. What is your name?'
prompt = PromptTemplate(template=prompt_template, input_variables=['name'])
prompt_str = prompt.format(name='John')
llm = OpenAI(verbose=False, cache=False)
output = llm(prompt_str)
print(output)

AttributeError: module 'langchain' has no attribute 'llm_cache'

Albiet, 1-3 could be combined with f strings, but still, that's a lot of steps for developers to learn. Now, see how the concise API makes prompting much more concise:

In [11]:
from langchain.concise import generate

output = generate("What is the capital of {country}?")

ValidationError: 1 validation error for ChatMessagePromptTemplate
role
  field required (type=value_error.missing)

For many use cases, the concise API is all you need. However, if you need more control, you can use the full API.

## Must-learn: `generate`

If there's one function in LangChain's concise API that you must learn, it's `generate`. `generate` is the Swiss Army knife of text generation. It can be used to generate text, construct python objects, make decisions, and select options. Let's see how it works.

### Generating text

The simplest use case for `generate` is to generate text. Let's generate a sentence about a dog.

In [ ]:
generate("What has four legs and barks?")

As you can see, `generate` produced a sentence about a dog. It's that simple. No templates, no prompts, no 5 hours learning langchain abstractions. 

### Generating python objects

But what if we want to actually *generate* a dog? Well, we can do that too. Let's generate a dog. To do this, we'll need to first define a class for dogs.

In [12]:
from pydantic import BaseModel, Field

class Dog(BaseModel):
    name: str = Field(..., description="Name of the dog")
    age: int = Field(..., description="Age of the dog")

Now all we have to do is pass the class to `generate` and it will generate a dog for us! Like so,

In [ ]:
generate("Make a dog named Spot that is 5 years old.", type=Dog)

You can use generate on any `pydantic` model. If you're not familiar with `pydantic`, it's a library for defining data models in python. It's used by FastAPI and many other libraries. You can learn more about it here: https://pydantic-docs.helpmanual.io/.

You can likewise generate `str`'s, `int`'s, `float`'s, and `bool`'s by passing the type to `generate`. For example, `generate("My name is Sam and I am 24 years old. How old am I?", type=int)` will generate an integer.

### Generating other objects

Not all LLM outputs are best parsed into pydantic models. For example, if we want to generate a list of dogs, we can't use a pydantic model because pydantic models are for single objects. However, we can manually instantiate an `ItemParsedListOutputParser` and pass that to the `parser` arg in `generate`. Let's see how that works.

In [14]:
from langchain.output_parsers.item_parsed_list import ItemParsedListOutputParser
from langchain.output_parsers.pydantic import PydanticOutputParser

dog_parser = PydanticOutputParser(pydantic_object=Dog)
dog_list_parser = ItemParsedListOutputParser(item_parser=dog_parser)

generate("Generate 3 dogs. You pick the names and ages.", parser=dog_list_parser)

NameError: name 'generate' is not defined

### Metaprompting

`generate`'s conciseness makes it very convenient for meta-prompting. Check it out.

In [16]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

age = None
chat_model = ChatOpenAI()

system_message = generate("Generate a persona statement. Begin with 'You are a...'.")
user_message = f"""Hello, my name is {generate("Generate a name.")}. I am {age := generate("Generate an age.", type=int)} years old. My sister is 5 years younger than me ({age-5} years old). I am a {generate("Generate a job title.")}. {generate("Write an 'I like to' statement. You can decide whatever it is.")}."""

chat_model([SystemMessage(content=system_message), HumanMessage(content=user_message)])

AttributeError: module 'langchain' has no attribute 'verbose'

In the above code, we generate a system persona on the fly. The simulated user generated several of their characteristics on the fly as well, including a sentence about the user's likes.

Perhaps a more practical example for LLM developers like yourself is using LLMs to generate prompts for other LLMs. Let's see how that works.

In [17]:
prompt = generate("You are a world-class LLM prompt engineer. Your prompts are almost always successful at generating the desired output. You are writing a prompt to generate a dog. You want to generate a dog named Spot that is 5 years old. You want to use the least amount of tokens possible. Write the prompt.")
print(prompt)
output = generate(prompt, type=Dog)
print(output)

NameError: name 'generate' is not defined

## Making decisions and selecting options

See how Langchain's concise API can intergrate deeper into you code with `decide` and `choice`. These functions return a boolean and an option respectively. Let's see how they work.

We'll start with `decide`. `decide` takes a query and a list of examples. It returns a boolean indicating whether the query is true or false.

In [19]:
from langchain.concise import decide


value = decide(prompt, query="Is this a good prompt?")
value

ValidationError: 1 validation error for ChatMessagePromptTemplate
role
  field required (type=value_error.missing)

Since its signature is so concise, you can integrate `decide` directly into your code. For example, you can use it to make decisions about whether to take a certain branch of code.

In [20]:
result = ''
while not decide(result, query="Is this a good story?"):
    result = generate('Write a short funny story')
print(result)

NameError: name 'decide' is not defined

Here's a natural language game:

In [ ]:
import ast
from langchain.output_parsers

position = (0,0)
map_size = (10,10)

while True:
    print(f'You are at position {position} on a map of size {map_size}. What do you do?')
    user = input(">>> ")
    if decide('Did the user ask a question?', input=user):
        print(generate(f"Answer the user's question.\n\nUser: {user}\nAI: "))
    elif decide('Did the user make a move?', input=user):
        delta = static_eval(generate(f"Translate the natural language motion into a delta position in tuple format (dx, dy)\n\nNatural language: {user}\nTuple format: "))
        position = (position[0] + delta[0], position[1] + delta[1])
        position = (max(0, min(map_size[0], position[0])), max(0, min(map_size[1], position[1])))
    else:
        print(generate(f'Apolagize for not understanding the user.\n\nUser: {user}\nAI: '))

Similarly, `choice` gives the LLM the ability to select an option from a list of options.

In [21]:
from langchain.concise import choice


flavor = choice("Sam loves ice cream. He enjoy chocolate and vanilla, but he really really loves strawberry.", query="Pick Sams favorite color", options=['chocolate', 'vanilla', 'strawberry'])
print(flavor)

ValidationError: 1 validation error for ChatMessagePromptTemplate
role
  field required (type=value_error.missing)

And likewise for Enum's:

In [ ]:
from enum import Enum


class Color(Enum):
    red = "red"
    yellow = "yellow"
    green = "green"
    blue = "blue"

user_name = "Steeve"
user_dossier = "Steve is our 10 year loyal customer. He is a big fan of our product and has been a great advocate for us. He drives a red truck and loves to eat pizza."

match choice(f"What color product is {user_name} most likely to buy?", options=Color):
    case Color.red:
        ...
    case Color.yellow:
        ...
    case Color.green:
        ...
    case Color.blue:
        ...

## Templates and Gemplates

Now let's explore some of the more cutting edge features of the concise API. We'll start with templates. Templates are statefull text templators. They are useful for keeping track of pronouns when generating text, such as prompts for LLMs. Let's see how they work.

In [23]:
from langchain.concise.template import template


t = template("You are {role}GPT. You can {capabilities}. If you do not understand the user, you should {fallback}.")
print(t(role="chitchat", capabilities="chat about anything", fallback="apologize and ask for clarification"))
print(t(role="book", capabilities="discuss literature", fallback="recommend a book about the topic"))
print(t(role="scholar"))
print(t(fallback="apologize and ask for clarification"))

ValidationError: 1 validation error for ChatMessagePromptTemplate
role
  field required (type=value_error.missing)

As you see, the template keeps track of the pronouns so that the user only has to enter changes to the template. This is convenient in some cases.

Gemplates are similar to templates, but they are for semantic kernels. In computer science, a kernel is a function that transforms one data structure into another. In LangChain, a semantic kernel is a function that transforms one semantic structure into another. Let's see how they work.

In [24]:
from langchain.concise import gemplate


gem = gemplate("Rewrite this sentence in the style of {name}: {sentence}")

print(gem(name="Steve Jobs", sentence="Oh Romeo, Romeo, wherefore art thou Romeo?"))
print(gem(name="Elon Musk"))

SyntaxError: invalid syntax (560798519.py, line 7)

## Rulex: Semantic pattern matching and replacement

Think regex, but with natural language. Rulex is a class for performing replacements on natural language with rules written in natural language.

Start by defining the rules:

In [ ]:
from langchain.concise.rulex import Rule


rules = [
    Rule(name="add comments", pattern="A block of code without any comments", replacement="The same code but with comments"),
    Rule(name="elaborate on comments", pattern="All comments", replacement="The same comment but explained with more detail"),
    Rule(name="decompose list comprehensions", pattern="List comprehension", replacement="The overall semantics of the list comprehension, but using a for loop"),
]

Hopefully, these rules will be self-explanatory. Now, let's use them to make some code more readable.

Here's the complex code:

In [ ]:
from langchain.concise.config import get_default_max_tokens
from langchain.output_parsers.code import CodeOutputParser
from langchain.output_parsers.incomplete import IncompleteOutputParser


code = generate("Generate a long complex python script that has very few comments and uses lots of list comprehensions. Answer inside a triple backtick code block.", parser=IncompleteOutputParser(parser=CodeOutputParser(), llm=get_default_max_tokens())
print(code)

And here's the simple code:

In [25]:
from langchain.concise.rulex import RulEx


ru = RulEx(rules=rules)

simplified_code = ru(code)

print(simplified_code)

ValidationError: 1 validation error for ChatMessagePromptTemplate
role
  field required (type=value_error.missing)

## Retaining flexbillity

Even though the `concise` API is, well, concise, it's still flexible. For example, you can pass a custom `LLM` or `TextSplitter` to any function that uses it. You can also change the default `LLM`, `TextSplitter`, and max tokens in `langchain.concise.config`.